In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
import copy
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import networkx as nx
from torch.nn.utils import parameters_to_vector, vector_to_parameters

In [2]:
# Graph implementation
def generate_graph(cluster_sizes=[100,100], pin=0.5, pout=0.01, seed=0):
    """Generate a random connected graph"""
    probs = np.array([[pin, pout],[pout, pin]])
    while True:
        g = nx.stochastic_block_model(cluster_sizes, probs)
        if nx.algorithms.components.is_connected(g):
            return g


cluster_sizes = [10, 10]
pin = 0.5
pout = 0.2
seed = 0
alpha = 1e-2
lamda = 1e-3
eta = 1e-2
no_users = sum(cluster_sizes)
batch_size = 20
epochs = 1
it = 2000
G = generate_graph(cluster_sizes, pin, pout, seed)

#nx.draw(G, with_labels=True, node_size=100, alpha=1, linewidths=10)
#plt.show()

In [3]:
# Metropolis weights 
number_nodes = G.number_of_nodes()
weights = np.zeros([number_nodes, number_nodes])
for edge in G.edges():
  i, j = edge[0], edge[1]
  weights[i - 1][j - 1] = 1 / (1 + np.max([G.degree(i), G.degree(j)]))
  weights[j - 1][i - 1] = weights[i - 1][j - 1]

print(weights)

weights = weights + np.diag(1 - np.sum(weights, axis=0))

metropolis_weights = weights
print(metropolis_weights)


[[0.         0.         0.11111111 0.11111111 0.11111111 0.11111111
  0.         0.11111111 0.         0.         0.         0.
  0.         0.11111111 0.11111111 0.11111111 0.         0.
  0.         0.        ]
 [0.         0.         0.14285714 0.         0.125      0.14285714
  0.14285714 0.         0.         0.         0.14285714 0.
  0.         0.         0.         0.         0.11111111 0.
  0.         0.        ]
 [0.11111111 0.14285714 0.         0.         0.125      0.
  0.14285714 0.         0.         0.         0.         0.1
  0.         0.         0.         0.         0.11111111 0.
  0.         0.        ]
 [0.11111111 0.         0.         0.         0.         0.
  0.         0.16666667 0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.1
  0.         0.125     ]
 [0.11111111 0.125      0.125      0.         0.         0.
  0.125      0.         0.         0.         0.         0.1
  0.         0.         0.125      0.    

In [4]:
def load_dataset():
    transforms_mnist = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,),(0.3081,))])
    mnist_data_train = datasets.MNIST('./data/mnist', train=True, download=True, transform=transforms_mnist)
    mnist_data_test = datasets.MNIST('./data/mnist', train=False, download=True, transform=transforms_mnist)

    return mnist_data_train, mnist_data_test

In [5]:
def degrees(A):
    """Return the degrees of each node of a graph from its adjacency matrix"""
    return np.sum(A, axis=0).reshape(A.shape[0], 1)

def node_degree(n, G):
    cnt = 0
    for i in G.neighbors(n):
        cnt += 1
    return cnt

def get_neighbors(n, G):
    neighbors_list = []
    for i in G.neighbors(n):
        neighbors_list.append(int(i))
    return neighbors_list

In [6]:
datapoints = {}
count = 0
W1 = np.array([2.0, 2.0])
W2 = np.array([-2.0, 2.0])
W = [W1, W2]
m = 200
n = 2
noise_sd = 0.001
for i, cluster_size in enumerate(cluster_sizes):
    for j in range(cluster_size):
        features = np.random.normal(loc=0.0, scale=1.0, size=(m, n))
        label = np.dot(features, W[i]) + np.random.normal(0,noise_sd)
        datapoints[count] = {
                'features': features,
                'degree': node_degree(count, G),
                'label': label,
                'neighbors': get_neighbors(count, G),
                'exact_weights': torch.from_numpy(W[i])
            }
        count += 1

In [7]:
class MyDataset(Dataset):
    def __init__(self, data, targets, transform=None):
        self.data = torch.FloatTensor(data)
        self.targets = torch.FloatTensor(targets).unsqueeze(-1)
        
    def __getitem__(self, index):
        x = self.data[index]
        y = self.targets[index]

        return x, y
    
    def __len__(self):
        return len(self.data)


In [8]:
class MLP_Net(nn.Module):
    def __init__(self, user_id):
        super(MLP_Net, self).__init__()
        self.fc1 = nn.Linear(2, 1, bias=False)
        #self.fc2 = nn.Linear(4, 1, bias=False)
        #self.fc3 = nn.Linear(200, 10)
        self.user_id = user_id

    def forward(self, x):
        x = torch.flatten(x, 1)
        #x = F.relu(self.fc1(x))
        output = self.fc1(x)
        #output = self.fc3(x)
        return output

In [9]:
from typing import Iterable, Optional

def grads_to_vector(parameters: Iterable[torch.Tensor]) -> torch.Tensor:
    r"""Convert parameters to one vector

    Args:
        parameters (Iterable[Tensor]): an iterator of Tensors that are the
            parameters of a model.

    Returns:
        The parameters represented by a single vector
    """
    # Flag for the device where the parameter is located
    param_device = None

    vec = []
    for param in parameters:
        # Ensure the parameters are located in the same device
        param_device = param.grad

        vec.append(param_device.view(-1))
    return torch.cat(vec)

In [10]:
model = MLP_Net(user_id=0)

lr = 0.01

dataloader = DataLoader(MyDataset(datapoints[19]["features"], datapoints[19]["label"]), batch_size=50, shuffle=False)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
for i in range(100):
    for (x, y) in dataloader:
        criterion = nn.MSELoss()
        optimizer.zero_grad()
        yhat = model(x)
        print(y.size())
        print(yhat.size())
        loss = criterion(yhat, y)
        
        loss.backward()
        print(i, loss, grads_to_vector(model.parameters()), parameters_to_vector(model.parameters()))
        #optimizer.step()
        new_model = parameters_to_vector(model.parameters()) - lr * grads_to_vector(model.parameters())
        vector_to_parameters(parameters=model.parameters(), vec=new_model)
        #if i % 50 ==0:
            #lr *= 0.9
            

#parameters_to_vector(model.parameters())

torch.Size([50, 1])
torch.Size([50, 1])
0 tensor(5.5483, grad_fn=<MseLossBackward0>) tensor([ 1.5745, -3.6494]) tensor([-0.3737, -0.3390], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
0 tensor(5.0687, grad_fn=<MseLossBackward0>) tensor([ 2.1480, -2.9002]) tensor([-0.3894, -0.3025], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
0 tensor(11.0110, grad_fn=<MseLossBackward0>) tensor([ 5.4639, -5.8672]) tensor([-0.4109, -0.2735], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
0 tensor(4.6273, grad_fn=<MseLossBackward0>) tensor([ 1.9390, -2.8354]) tensor([-0.4656, -0.2148], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
1 tensor(4.8406, grad_fn=<MseLossBackward0>) tensor([ 1.4647, -3.4129]) tensor([-0.4850, -0.1865], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
1 tensor(4.4189, grad_fn=<MseLossBackward0>) tensor([ 1.9993, -2.7123]) tensor([-0.4996, -0.1523], grad_fn=<CatBackward0>)
torch.Size([50, 1])
t

torch.Size([50, 1])
torch.Size([50, 1])
14 tensor(0.6797, grad_fn=<MseLossBackward0>) tensor([ 0.7059, -1.1120]) tensor([-1.4301,  1.1390], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
15 tensor(0.7170, grad_fn=<MseLossBackward0>) tensor([ 0.5336, -1.3339]) tensor([-1.4372,  1.1501], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
15 tensor(0.6482, grad_fn=<MseLossBackward0>) tensor([ 0.7340, -1.0605]) tensor([-1.4425,  1.1634], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
15 tensor(1.4004, grad_fn=<MseLossBackward0>) tensor([ 1.9126, -2.1170]) tensor([-1.4499,  1.1740], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
15 tensor(0.5928, grad_fn=<MseLossBackward0>) tensor([ 0.6568, -1.0400]) tensor([-1.4690,  1.1952], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
16 tensor(0.6256, grad_fn=<MseLossBackward0>) tensor([ 0.4966, -1.2472]) tensor([-1.4756,  1.2056], grad_fn=<CatBackward0>)
torch.Size([50, 

27 tensor(0.1253, grad_fn=<MseLossBackward0>) tensor([ 0.3120, -0.4733]) tensor([-1.7608,  1.6282], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
27 tensor(0.2695, grad_fn=<MseLossBackward0>) tensor([ 0.8276, -0.9362]) tensor([-1.7640,  1.6329], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
27 tensor(0.1147, grad_fn=<MseLossBackward0>) tensor([ 0.2774, -0.4649]) tensor([-1.7722,  1.6423], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
28 tensor(0.1218, grad_fn=<MseLossBackward0>) tensor([ 0.2098, -0.5564]) tensor([-1.7750,  1.6470], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
28 tensor(0.1093, grad_fn=<MseLossBackward0>) tensor([ 0.2906, -0.4425]) tensor([-1.7771,  1.6525], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
28 tensor(0.2349, grad_fn=<MseLossBackward0>) tensor([ 0.7718, -0.8746]) tensor([-1.7800,  1.6569], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
28 tensor(0.1000

torch.Size([50, 1])
torch.Size([50, 1])
42 tensor(0.0344, grad_fn=<MseLossBackward0>) tensor([ 0.2911, -0.3375]) tensor([-1.9179,  1.8669], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
42 tensor(0.0147, grad_fn=<MseLossBackward0>) tensor([ 0.0949, -0.1694]) tensor([-1.9208,  1.8703], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
43 tensor(0.0158, grad_fn=<MseLossBackward0>) tensor([ 0.0718, -0.2025]) tensor([-1.9217,  1.8720], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
43 tensor(0.0140, grad_fn=<MseLossBackward0>) tensor([ 0.1002, -0.1610]) tensor([-1.9224,  1.8740], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
43 tensor(0.0300, grad_fn=<MseLossBackward0>) tensor([ 0.2715, -0.3153]) tensor([-1.9234,  1.8756], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
43 tensor(0.0128, grad_fn=<MseLossBackward0>) tensor([ 0.0883, -0.1584]) tensor([-1.9262,  1.8788], grad_fn=<CatBackward0>)
torch.Size([50, 

torch.Size([50, 1])
torch.Size([50, 1])
55 tensor(0.0058, grad_fn=<MseLossBackward0>) tensor([ 0.1178, -0.1394]) tensor([-1.9671,  1.9447], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
55 tensor(0.0025, grad_fn=<MseLossBackward0>) tensor([ 0.0376, -0.0704]) tensor([-1.9683,  1.9461], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
56 tensor(0.0027, grad_fn=<MseLossBackward0>) tensor([ 0.0284, -0.0842]) tensor([-1.9687,  1.9468], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
56 tensor(0.0024, grad_fn=<MseLossBackward0>) tensor([ 0.0400, -0.0670]) tensor([-1.9690,  1.9477], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
56 tensor(0.0051, grad_fn=<MseLossBackward0>) tensor([ 0.1099, -0.1302]) tensor([-1.9694,  1.9484], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
56 tensor(0.0022, grad_fn=<MseLossBackward0>) tensor([ 0.0351, -0.0658]) tensor([-1.9705,  1.9497], grad_fn=<CatBackward0>)
torch.Size([50, 

68 tensor(0.0005, grad_fn=<MseLossBackward0>) tensor([ 0.0121, -0.0375]) tensor([-1.9866,  1.9763], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
68 tensor(0.0005, grad_fn=<MseLossBackward0>) tensor([ 0.0172, -0.0298]) tensor([-1.9867,  1.9767], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
68 tensor(0.0010, grad_fn=<MseLossBackward0>) tensor([ 0.0477, -0.0576]) tensor([-1.9869,  1.9770], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
68 tensor(0.0004, grad_fn=<MseLossBackward0>) tensor([ 0.0150, -0.0291]) tensor([-1.9874,  1.9776], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
69 tensor(0.0005, grad_fn=<MseLossBackward0>) tensor([ 0.0113, -0.0351]) tensor([-1.9875,  1.9779], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
69 tensor(0.0004, grad_fn=<MseLossBackward0>) tensor([ 0.0161, -0.0278]) tensor([-1.9876,  1.9782], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
69 tensor(0.0009

82 tensor(7.1845e-05, grad_fn=<MseLossBackward0>) tensor([ 0.0065, -0.0116]) tensor([-1.9952,  1.9909], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
82 tensor(0.0001, grad_fn=<MseLossBackward0>) tensor([ 0.0180, -0.0223]) tensor([-1.9952,  1.9910], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
82 tensor(6.3238e-05, grad_fn=<MseLossBackward0>) tensor([ 0.0056, -0.0111]) tensor([-1.9954,  1.9912], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
83 tensor(7.1765e-05, grad_fn=<MseLossBackward0>) tensor([ 0.0042, -0.0137]) tensor([-1.9955,  1.9913], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
83 tensor(6.3130e-05, grad_fn=<MseLossBackward0>) tensor([ 0.0061, -0.0108]) tensor([-1.9955,  1.9915], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
83 tensor(0.0001, grad_fn=<MseLossBackward0>) tensor([ 0.0168, -0.0208]) tensor([-1.9956,  1.9916], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])


98 tensor(1.9377e-05, grad_fn=<MseLossBackward0>) tensor([ 0.0058, -0.0076]) tensor([-1.9986,  1.9968], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
98 tensor(9.3351e-06, grad_fn=<MseLossBackward0>) tensor([ 0.0019, -0.0035]) tensor([-1.9987,  1.9969], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
99 tensor(1.1216e-05, grad_fn=<MseLossBackward0>) tensor([ 0.0013, -0.0047]) tensor([-1.9987,  1.9969], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
99 tensor(1.0093e-05, grad_fn=<MseLossBackward0>) tensor([ 0.0021, -0.0037]) tensor([-1.9987,  1.9970], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
99 tensor(1.7321e-05, grad_fn=<MseLossBackward0>) tensor([ 0.0054, -0.0071]) tensor([-1.9987,  1.9970], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
99 tensor(8.4893e-06, grad_fn=<MseLossBackward0>) tensor([ 0.0018, -0.0033]) tensor([-1.9988,  1.9971], grad_fn=<CatBackward0>)


In [11]:
parameters_to_vector(model.parameters())

tensor([-1.9988,  1.9971], grad_fn=<CatBackward0>)

In [12]:
class CNN_Net(nn.Module):
    def __init__(self):
        super(CNN_Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=5)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=5)
        self.pool = nn.MaxPool2d(2,2)
        self.dropout = nn.Dropout(p=0.2)
        self.fc1 = nn.Linear(1024, 512)
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.dropout(x)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        output = self.fc2(x)
        return output

In [13]:
class ClientUpdate(object):
    def __init__(self, dataset, batchSize, alpha, lamda, epochs, projection_list, projected_weights):
        self.train_loader = DataLoader(MyDataset(dataset["features"], dataset["label"]), batch_size=batchSize, shuffle=True)
        #self.learning_rate = learning_rate
        self.epochs = epochs
        self.batchSize = batchSize

    def train(self, model):
        criterion = nn.MSELoss()
        optimizer = torch.optim.SGD(model.parameters(), lr=1e-3, momentum=0.5)

        e_loss = []
        for epoch in range(1, self.epochs+1):
            train_loss = 0
            model.train()
            for i, (data, labels) in zip(range(1), self.train_loader):
                data, labels = data, labels
                optimizer.zero_grad() 
                output = model(data)  
                loss = criterion(output, labels)
                #loss += mu/2 * torch.norm(client_param.data - server_param.data)**2
                loss.backward()
                grads = grads_to_vector(model.parameters())
                #optimizer.step()
                train_loss += loss.item()*data.size(0)
                weights = parameters_to_vector(model.parameters())
                mat_vec_sum = torch.zeros_like(weights)
                for j in G.neighbors(model.user_id):
                    mat_vec_sum = torch.add(mat_vec_sum, torch.matmul(torch.transpose(projection_list[model.user_id][j], 0, 1), 
                                                         projected_weights[j][model.user_id] - projected_weights[model.user_id][j]))
                
                model_update = parameters_to_vector(model.parameters()) - alpha * (grads + lamda * mat_vec_sum)
                
            vector_to_parameters(parameters=model.parameters(), vec=model_update)
                

            train_loss = train_loss/self.batchSize#len(self.train_loader.dataset) 
            e_loss.append(train_loss)

        total_loss = e_loss#sum(e_loss)/len(e_loss)

        return model.state_dict(), total_loss

In [14]:
# Preparing projection matrices
models = [MLP_Net(user_id=i) for i in range(no_users)]
#temp = MLP_Net()
projection_list = []
projected_weights = []

def update_ProjWeight(projection_list, projected_weights, first_run=True):
    #projected_weights = []
    for i in range(no_users):
        neighbors_mat = []
        neighbors_weights = []
        for j in range(no_users):
            if j in G.neighbors(i):
                with torch.no_grad():
                    if first_run == True:
                        row, column = parameters_to_vector(models[j].parameters()).size()[0], parameters_to_vector(models[i].parameters()).size()[0]
                        mat = torch.zeros((row, column))
                        mat.fill_diagonal_(1.0 + 1.0 * float(np.random.randn(1)))
                        neighbors_mat.append(mat)
                        neighbors_weights.append(torch.matmul(mat, parameters_to_vector(models[j].parameters())))
                    else:
                        neighbors_weights.append(torch.matmul(projection_list[j][i], parameters_to_vector(models[j].parameters())))
            else:
                neighbors_mat.append(0)
                neighbors_weights.append(0)
        if first_run == True:
            projection_list.append(neighbors_mat)
        projected_weights.append(neighbors_weights)

update_ProjWeight(projection_list, projected_weights)



In [15]:
print(projection_list[0])

[0, 0, 0, 0, tensor([[1.0796, 0.0000],
        [0.0000, 1.0796]]), tensor([[0.2804, 0.0000],
        [0.0000, 0.2804]]), tensor([[1.8262, 0.0000],
        [0.0000, 1.8262]]), tensor([[2.2648, 0.0000],
        [0.0000, 2.2648]]), tensor([[1.2421, 0.0000],
        [0.0000, 1.2421]]), tensor([[0.3798, 0.0000],
        [0.0000, 0.3798]]), 0, 0, tensor([[-0.0181,  0.0000],
        [ 0.0000, -0.0181]]), 0, 0, 0, 0, 0, 0, 0]


In [16]:
def testing(model, dataset, bs, criterion): 
    test_loss = 0
    correct = 0
    test_loader = DataLoader(MyDataset(dataset["features"], dataset["label"]), batch_size=bs)
    l = len(test_loader)
    model.eval()
    for data, labels in test_loader:
        data, labels = data, labels
        output = model(data)
        loss = criterion(output, labels)
        test_loss += loss.item()*data.size(0)
        #_, pred = torch.max(output, 1)
        #correct += pred.eq(labels.data.view_as(pred)).sum().item()
    
    test_loss /= len(test_loader.dataset)
    
    return test_loss

In [17]:
def rel_error(model):
    return (torch.norm(parameters_to_vector(model.parameters()) - datapoints[model.user_id]['exact_weights']) / torch.norm(datapoints[model.user_id]['exact_weights'])).detach()

In [18]:
torch.norm(datapoints[model.user_id]['exact_weights'])

tensor(2.8284, dtype=torch.float64)

In [19]:
model = MLP_Net(user_id=0)

from torch.nn.utils import parameters_to_vector, vector_to_parameters

with torch.no_grad():    
    params = parameters_to_vector(model.parameters())

    print(params)

params *= 2.

vector_to_parameters(parameters=model.parameters(), vec=params)

parameters_to_vector(model.parameters())





tensor([ 0.0320, -0.0879])


tensor([ 0.0640, -0.1758], grad_fn=<CatBackward0>)

In [20]:
#global_model = CNN_Net().cuda()
models = [MLP_Net(user_id=i) for i in range(no_users)]
dummy_models = [MLP_Net(user_id=i) for i in range(no_users)]

#model.load_state_dict(global_model.state_dict())

criterion = nn.MSELoss()


train_loss = []
test_loss = []
test_accuracy = []
total_rel_error = []

for curr_round in tqdm(range(1, it+1)):
    w, local_loss = [], []

    
    for i in range(no_users):
        dummy_models[i].load_state_dict(models[i].state_dict())
        local_update = ClientUpdate(dataset=datapoints[i], batchSize=batch_size, alpha=alpha, lamda=lamda, epochs=1, projection_list=projection_list, projected_weights=projected_weights)
        weights, loss = local_update.train(dummy_models[i])
        w.append(weights)
        local_loss.append(loss)
        models[i].load_state_dict(w[i])
        
    
    
    # Update prjection matrix
    
    #print(projection_list[0], projected_weights[0])
    
    for i in range(no_users):
        weights = parameters_to_vector(models[i].parameters())
        for j in G.neighbors(i):
            mat_vec_sum = torch.zeros_like(weights)
            for k in G.neighbors(i):
                mat_vec_sum = torch.add(mat_vec_sum, projected_weights[k][i] - projected_weights[i][k])
            #print(torch.outer(mat_vec_sum, weights))

            projection_list[i][j] = torch.add(projection_list[i][j], -1 * eta * lamda * mat_vec_sum)
                                         
    projected_weights = []                                          
    update_ProjWeight(projection_list, projected_weights, first_run=False)
        
        
        
    
    




          
            

    local_test_acc = []
    local_test_loss = []
    user_rel_error = 0
    for k in range(no_users):
      
        g_loss = testing(models[i], datapoints[i], 50, criterion)
        local_test_loss.append(g_loss)
        user_rel_error += rel_error(models[i])
    
    
        

    g_loss = sum(local_test_loss) / len(local_test_loss)
    total_rel_error.append(user_rel_error / no_users)
    
    

    test_loss.append(g_loss)
    #test_accuracy.append(g_accuracy)
    print("Training_loss %2.5f,   Relative Error %2.5f"% (test_loss[-1], total_rel_error[-1]))

  0%|          | 2/2000 [00:00<06:08,  5.43it/s]

Training_loss 10.73425,   Relative Error 1.25024
Training_loss 10.33293,   Relative Error 1.22652


  0%|          | 4/2000 [00:00<05:39,  5.88it/s]

Training_loss 9.99157,   Relative Error 1.20640
Training_loss 9.56407,   Relative Error 1.18045


  0%|          | 5/2000 [00:00<05:51,  5.68it/s]

...

Training_loss 0.00001,   Relative Error 0.00094
Training_loss 0.00001,   Relative Error 0.00093


100%|█████████▉| 1999/2000 [04:14<00:00,  6.51it/s]

Training_loss 0.00001,   Relative Error 0.00093
Training_loss 0.00001,   Relative Error 0.00093


100%|██████████| 2000/2000 [04:15<00:00,  7.84it/s]

Training_loss 0.00001,   Relative Error 0.00094


In [21]:
#plot.plot(test_loss)
parameters_to_vector(models[19].parameters())

tensor([-1.9978,  2.0015], grad_fn=<CatBackward0>)

In [22]:
for j in G.neighbors(0):
    print(j)

4
5
6
7
8
9
12


In [23]:
parameters_to_vector(models[0].parameters())

tensor([1.9953, 1.9952], grad_fn=<CatBackward0>)

In [24]:
projection_list[0]

[0,
 0,
 0,
 0,
 tensor([[ 1.0122, -0.1058],
         [-0.0674,  0.9738]]),
 tensor([[ 0.2129, -0.1058],
         [-0.0674,  0.1746]]),
 tensor([[ 1.7588, -0.1058],
         [-0.0674,  1.7205]]),
 tensor([[ 2.1973, -0.1058],
         [-0.0674,  2.1590]]),
 tensor([[ 1.1746, -0.1058],
         [-0.0674,  1.1363]]),
 tensor([[ 0.3124, -0.1058],
         [-0.0674,  0.2741]]),
 0,
 0,
 tensor([[-0.0856, -0.1058],
         [-0.0674, -0.1239]]),
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [25]:
projected_weights[0]

[0,
 0,
 0,
 0,
 tensor([1.0228, 1.0229]),
 tensor([3.7029, 3.6958]),
 tensor([0.9197, 0.9219]),
 tensor([2.0308, 2.0312]),
 tensor([0.3476, 0.3476]),
 tensor([0.4399, 0.4417]),
 0,
 0,
 tensor([ 0.4205, -1.5137]),
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [26]:
test_loss = np.array(test_loss)
total_rel_error = np.array(total_rel_error)

In [27]:
print(test_loss)

[1.07342498e+01 1.03329268e+01 9.99157465e+00 ... 1.16781121e-05
 1.17123579e-05 1.17857328e-05]


In [28]:
np.save( 'training_loss_sheave_fml' + str(eta).replace('.', '_')+ '_pout' + str(pout).replace('.', '_'), test_loss)
np.save('relative_error_sheave_fml' + str(eta).replace('.', '_')+ '_pout' + str(pout).replace('.', '_'), total_rel_error)

In [29]:
'training_loss_sheave_fml' + str(eta).replace('.', '_')

'training_loss_sheave_fml0_01'

In [30]:
'relative_error_sheave_fml' + str(eta).replace('.', '_')

'relative_error_sheave_fml0_01'